In [ ]:
%env CUDA_VISIBLE_DEVICES=1
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")

In [ ]:
import torch

distances_dino_diff = torch.load("../analysis_data/distances_dino_diff.pt")
distances_diff = torch.load("../analysis_data/distances_diff.pt")
distances_dino = torch.load("../analysis_data/distances_dino.pt")
distances_dino_s = torch.load("../analysis_data/distances_dino_s.pt")
#distances_clip = torch.load("../analysis_data/distances_clip.pt")

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))

x_thresh = 0.1 * 768

max = 1000
bins = 100
print(distances_dino_diff.max(), distances_dino_diff.min())
#dino_diff_hist = torch.histc(distances_dino_diff.float(), bins=bins, min=0, max=max)
diff_hist = torch.histc(distances_diff.float(), bins=bins, min=0, max=max)
dino_hist = torch.histc(distances_dino.float(), bins=bins, min=0, max=max)
#dino_hist_s = torch.histc(distances_dino_s.float(), bins=bins, min=0, max=max)
#clip_hist = torch.histc(distances_clip.float(), bins=100, min=0, max=max)

plt.plot(diff_hist, label="ADD-XL")
plt.plot(dino_hist, label="DINOv2 B/14")
#plt.plot(dino_diff_hist, label="DINOv2 B/14 and ADD-XL")
#plt.plot(dino_hist_s, label="DINOv2 B/14 336x336")
#plt.plot(clip_hist, label="CLIP L/14")

# draw vertical line at threshold
plt.axvline(x = (x_thresh / max) * bins, color = 'r', label = 'img 0.1 threshold')

print(diff_hist[:int((x_thresh / max) * bins)].sum() / diff_hist.sum())
print(dino_hist[:int((x_thresh / max) * bins)].sum() / dino_hist.sum())

plt.legend()
plt.show()

In [ ]:
similarity_map_hists_diff = torch.load("../analysis_data/histograms_diff.pt")
similarity_map_hists_dino = torch.load("../analysis_data/histograms_dino.pt")
similarity_map_hists_dino_diff = torch.load("../analysis_data/histograms_dino_diff.pt")
similarity_map_hists_dino_s = torch.load("../analysis_data/histograms_dino_s.pt")

plt.figure(figsize=(10, 5))
plt.plot(similarity_map_hists_diff, label="ADD-XL")
plt.plot(similarity_map_hists_dino, label="DINOv2 B/14")
plt.plot(similarity_map_hists_dino_diff, label="DINOv2 B/14 and ADD-XL")
plt.plot(similarity_map_hists_dino_s, label="DINOv2 B/14 336x336")
plt.legend()
plt.show()

In [ ]:
# Aggregate correct part and subpart predictions
# Visualize the results

from datasets.correspondence import S2K

dataset = S2K({
    "path": "/export/group/datasets/PASCAL-Part",
})